The `vasp_data` module of `surfaxe` contains scripts that collate the data from final VASP calculations for post-processing, and extract core and vacuum energy levels for band alignmnent diagrams. The latter two functions are fairly flexible and can be quickly used from the command line directly on the HPCs but have been integrated into final data collation as well. 

In [8]:
from surfaxe.vasp_data import process_data, core_energy, vacuum

# Create a path to the examples folder
from pathlib import Path 
path_to_vasp_data = Path.cwd().parents[1].joinpath('example_data/vasp_data')

### Vacuum level potential 
The vacuum level potential from a `LOCPOT` or an existing `potential.csv` file made with `analysis.potential` function can be extracted with aptly named `vacuum` function. It takes one required argument, `path` which is the path to the directory in which either of the files is. It first checks if a `potential.csv` exists, and if not it checks if a `LOCPOT` exists. If neither exist it raises a UserWarning. 

In [12]:
# Create a string of a path to the potential.csv file
path_to_potential = str(path_to_vasp_data.joinpath('101/potential.csv'))

vacuum(path_to_potential)

7.926

### Core state energy 
The `core_energy` function parses a structure file and an `OUTCAR` to get the core level of an atom in the middle of the slab with a specified coordination environment. It needs two parameters to work, the core atom and the list of nearest neighbours. The default nearest neighbour finding method is `CrystalNN` and as always, make sure to test the method works on the bulk structure before using it on slabs. If a different nearest neighbour algorithm is needed, it must first be imported from `pymatgen.analysis.local_env`  

In [13]:
outcar_path = str(path_to_vasp_data.joinpath('101/OUTCAR'))
structure_path = str(path_to_vasp_data.joinpath('101/POSCAR'))

core_energy('O', ['Sn', 'Sn', 'Sn'], outcar=outcar_path, structure=structure_path)

-504.2464

### Data collation
`process_data` function is only acessible through a python API as its use is more suitable for the collation of data from final set of VASP calculations. It requries the calculations to be in a specific folder structure, with one folder corresponding to the final caluclation for one Miller index. The function requries the energy per atom from a converged bulk calculation to calculate the surface energy but it can parse out the Miller indices from folder names. 

Example folder structure: 
(figure out how to insert figures in markdown) 

The function can also optionally make use of `core_energy` and `vacuum` functions to obtain the core energy levels and vacuum levels from the calculations along with the energies and input parameters. Because both require certain INCAR tags to be set manually, the `parse_core_energy` and `parse_vacuum` are set to `False` by default. All `core_energy` keyword arguments can be passed to `process_data` for full customisability. 

`process_data` parses the input parameteres part of `vasprun.xml` for the number of atoms and the area of the slab, the `KPOINTS` and the functional used and the following `INCAR` tags: `ENCUT`, `ISMEAR`, `SIGMA`, `ALGO`. From the output part of `vasprun.xml`, band gap, total energy and total energy per atom are collected. Based on the bulk energy per atom provided to the function, the surface energy is calculated. Lastly, it can either return a pandas DataFrame or save it to a csv file. 

In [14]:
process_data(-6.6118, parse_core_energy=True, path_to_fols=str(path_to_vasp_data),
             core_atom='O', bulk_nn=['Sn', 'Sn', 'Sn'], parse_vacuum=True, save_csv=Fals)

/anaconda3/lib/python3.7/site-packages/pymatgen/io/vasp/outputs.py:889: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/katarinabrlec/Documents/PhD/surfaxe/example_data/vasp_data/110
  " was found in {}".format(os.path.abspath(p)))
/anaconda3/lib/python3.7/site-packages/pymatgen/io/vasp/outputs.py:889: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/katarinabrlec/Documents/PhD/surfaxe/example_data/vasp_data/101
  " was found in {}".format(os.path.abspath(p)))


,hkl,hkl_tuple,area,atoms,functional,encut,algo,ismear,sigma,kpoints,bandgap,slab_energy,slab_per_atom,surface_energy,vacuum_potential,core_energy
0,110,"(1, 1, 0)",21.708192,72,PBESol,550.0,Normal,0,0.02,"[[6, 3, 1]]",0.5914,-472.864305,-6.567560,1.175327,9.077,-503.5028
1,101,"(1, 0, 1)",27.468479,72,PBESol,550.0,Normal,0,0.02,"[[4, 3, 1]]",0.7203,-470.608146,-6.536224,1.586766,7.926,-504.2464
